In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
import requests
import os
import matplotlib.pyplot as plt

In [8]:
from dotenv import load_dotenv
import os
load_dotenv()  # reads the .env file automatically
api_key = os.getenv("EIA_KEY")

26TPKw90CblDYys6OptSBsJH6jerRkeBvDOaxlVl


In [17]:
url = "https://api.eia.gov/v2/electricity/rto/fuel-type-data/data/?api_key=26TPKw90CblDYys6OptSBsJH6jerRkeBvDOaxlVl&frequency=hourly&data[0]=value&facets[fueltype][]=BAT&facets[fueltype][]=BAT&facets[fueltype][]=COL&facets[fueltype][]=GEO&facets[fueltype][]=NG&facets[fueltype][]=NUC&facets[fueltype][]=OES&facets[fueltype][]=OIL&facets[fueltype][]=OTH&facets[fueltype][]=PS&facets[fueltype][]=PS&facets[fueltype][]=SNB&facets[fueltype][]=SNB&facets[fueltype][]=SUN&facets[fueltype][]=UES&facets[fueltype][]=UES&facets[fueltype][]=UNK&facets[fueltype][]=WAT&facets[fueltype][]=WNB&facets[fueltype][]=WND&facets[respondent][]=CAL&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000"

In [20]:
data = requests.get(url).json()

In [21]:
data

{'warnings': [{'warning': 'incomplete return',
   'description': 'The API can only return 5000 rows in JSON format.  Please consider constraining your request with facet, start, or end, or using offset to paginate results.'}],
 'response': {'total': '497416',
  'dateFormat': 'YYYY-MM-DD"T"HH24',
  'frequency': 'hourly',
  'data': [{'period': '2025-09-26T06',
    'respondent': 'CAL',
    'respondent-name': 'California',
    'fueltype': 'BAT',
    'type-name': 'Battery storage',
    'value': '6',
    'value-units': 'megawatthours'},
   {'period': '2025-09-26T06',
    'respondent': 'CAL',
    'respondent-name': 'California',
    'fueltype': 'COL',
    'type-name': 'Coal',
    'value': '206',
    'value-units': 'megawatthours'},
   {'period': '2025-09-26T06',
    'respondent': 'CAL',
    'respondent-name': 'California',
    'fueltype': 'GEO',
    'type-name': 'Geothermal',
    'value': '436',
    'value-units': 'megawatthours'},
   {'period': '2025-09-26T06',
    'respondent': 'CAL',
    '

In [23]:
import os, requests, pandas as pd

API_KEY = os.getenv("EIA_KEY")  # set: export EIA_KEY="your_key"
BASE = "https://api.eia.gov/v2/electricity/rto/fuel-type-data/data/"

FUELTYPES = ["BAT","COL","GEO","NG","NUC","OES","OIL","OTH",
             "PS","SNB","SUN","UES","UNK","WAT","WNB","WND"]

COMMON = {
    "frequency": "hourly",
    "data[0]": "value",
    "facets[respondent][]": "CAL",
    "start": "2023-01-01T00",
    "end":   "2025-08-31T00",
    "sort[0][column]": "period",
    "sort[0][direction]": "desc",
    "length": 5000
}

def fetch_all_pages():
    all_rows, offset = [], 0
    while True:
        params = COMMON.copy()
        params["api_key"] = API_KEY
        params["offset"] = offset
        # add fuels
        for ft in FUELTYPES:
            params.setdefault("facets[fueltype][]", [])
            params["facets[fueltype][]"].append(ft)

        r = requests.get(BASE, params=params, timeout=60)
        r.raise_for_status()
        resp = r.json()["response"]

        rows = resp.get("data", [])
        all_rows.extend(rows)
        total = int(resp.get("total", 0))
        print(f"Fetched {len(rows):4d} rows @ offset {offset:,} "
              f"({len(all_rows):,}/{total:,})")

        # stop when last page (fewer than requested) or reached total
        if len(rows) < params["length"] or offset + params["length"] >= total:
            break
        offset += params["length"]

    return pd.DataFrame(all_rows)

df = fetch_all_pages()
print(df.shape)
print(df.head())
# Optional: save
# df.to_csv("eia_CAL_fueltype_2023_2025-08.csv", index=False)

Fetched 5000 rows @ offset 0 (5,000/209,352)
Fetched 5000 rows @ offset 5,000 (10,000/209,352)
Fetched 5000 rows @ offset 10,000 (15,000/209,352)
Fetched 5000 rows @ offset 15,000 (20,000/209,352)
Fetched 5000 rows @ offset 20,000 (25,000/209,352)
Fetched 5000 rows @ offset 25,000 (30,000/209,352)
Fetched 5000 rows @ offset 30,000 (35,000/209,352)
Fetched 5000 rows @ offset 35,000 (40,000/209,352)
Fetched 5000 rows @ offset 40,000 (45,000/209,352)
Fetched 5000 rows @ offset 45,000 (50,000/209,352)
Fetched 5000 rows @ offset 50,000 (55,000/209,352)
Fetched 5000 rows @ offset 55,000 (60,000/209,352)
Fetched 5000 rows @ offset 60,000 (65,000/209,352)
Fetched 5000 rows @ offset 65,000 (70,000/209,352)
Fetched 5000 rows @ offset 70,000 (75,000/209,352)
Fetched 5000 rows @ offset 75,000 (80,000/209,352)
Fetched 5000 rows @ offset 80,000 (85,000/209,352)
Fetched 5000 rows @ offset 85,000 (90,000/209,352)
Fetched 5000 rows @ offset 90,000 (95,000/209,352)
Fetched 5000 rows @ offset 95,000 (100

In [24]:
df.shape

(209352, 7)

In [25]:
df.head()

,period,respondent,respondent-name,fueltype,type-name,value,value-units
0,2025-08-31T00,CAL,California,BAT,Battery storage,11,megawatthours
1,2025-08-31T00,CAL,California,COL,Coal,202,megawatthours
2,2025-08-31T00,CAL,California,GEO,Geothermal,304,megawatthours
3,2025-08-31T00,CAL,California,NG,Natural Gas,13248,megawatthours
4,2025-08-31T00,CAL,California,NUC,Nuclear,2267,megawatthours
